# Forecast Cost and Usage Modeling

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
from datetime import datetime
import datetime as dt  # Used to manage dates
import warnings                   # To ignore the warnings warnings.filterwarnings("ignore")
from dateutil.parser import parse 
import pymysql # Import MySql 
import warnings                   # To ignore the warnings warnings.filterwarnings("ignore")
import requests

from scipy import signal
from scipy.special import inv_boxcox
from scipy import stats as stats # Used for calculating z-values

import numpy as np # Used for calculating statistics on data
import random
import math
from math import sqrt, log, exp
import itertools

import pandas as pd # Used for loading-in and viewing data
from pandas.plotting import lag_plot

import matplotlib.pyplot as plt # Used for creating graphics
import matplotlib as mpl
import matplotlib.lines as mlines
from matplotlib.pylab import rcParams 
from pylab import *
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import pylab as pl

import statsmodels.api as sm
#from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from statsmodels.tsa.stattools import adfuller, kpss, acf, pacf
from statsmodels.tsa.seasonal import seasonal_decompose 
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.formula.api import ols

#from statsmodels.tsa.arima_model import ARIMA, ARMA
#from statsmodels.tsa.arima_process import arma_generate_sample

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_squared_log_error
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import MinMaxScaler


import plotly.offline as py
from plotly import graph_objs as go
from plotly import figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots


from fbprophet.plot import add_changepoints_to_plot
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet.plot import plot_cross_validation_metric
from fbprophet import Prophet
from fbprophet.plot import plot_plotly


%matplotlib inline
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore", DeprecationWarning)
warnings.simplefilter("ignore", FutureWarning, )

pd.plotting.register_matplotlib_converters()
pd.options.display.float_format = '{:.4f}'.format
rcParams.update({'figure.autolayout': True, 'figure.figsize':(12,8),'axes.titlesize':14})

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('max_info_rows', 100)

# to make this notebook's output identical at every run
np.random.seed(42)

# Functions Definitions

In [4]:
def read_data(file_name):
    df = pd.read_csv(file_name, sep=',', index_col=False, header='infer', float_precision=2 )
    return df

# Load data transacton File View

* CREATE 
    ALGORITHM = UNDEFINED 
    DEFINER = `root`@`localhost` 
    SQL SECURITY DEFINER
VIEW `capstone_ai`.`household_ts_view` AS 
    SELECT 
        `a`.`call_status_recv_date` AS `call_date`,
        `a`.`membership_id` AS `membership_id`,
        `a`.`total_cost` AS `total_cost`,
        `a`.`csrd_yyyymm` AS `csrd_yyyymm`,
        `a`.`csrd_yyyy` AS `csrd_yyyy`,
        MONTH(`a`.`call_status_recv_date`) AS `csrd_mth`,
        `a`.`csrd_yyyywk` AS `csrd_yyyywk`,
     `capstone_ai`.`capstone_aaa_featured` `a`
    WHERE
        (`a`.`csrd_yyyy` <> 0)
    ORDER BY (`a`.`call_status_recv_date`
        AND `a`.`membership_id`)
        
    * call_status_recv_date - Call Date when the customer called AAA
    * membership_id - membership id - unique for the entire household
    * total_cost - Total cost of the service call
    * csrd_yyyymm - Year and month of the call date
    * csrd_yyyy - Year of the call date
    * csrd_mth - Month of the service call
    * csrd_yyyywk`- Year/week of the service call
    * csrd_wk - Week of the service call

In [5]:
# Load data from the View create using Capstone AAA - EDA .ipynb
df_household = read_data('household_ts_view.csv')
df_original_house_hold = df_household.copy()
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
corporacion = 'AAA Northeast'
var_to_predict = ['total_cost']
var_to_predict_title = 'ID Theft Services'
var_to_predict_save = 'theft'
test_size = 0.5

# To be Used to calculate the total number of calls 
df_household['no_call'] = 1

In [6]:
df_household.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13844 entries, 0 to 13843
Data columns (total 9 columns):
call_date        object
membership_id    int64
total_cost       float64
csrd_yyyymm      int64
csrd_yyyy        int64
csrd_mth         int64
csrd_yyyywk      int64
csrd_wk          int64
no_call          int64
dtypes: float64(1), int64(7), object(1)
memory usage: 973.5+ KB
